In [169]:
from collections import Counter
import random
import string
import Levenshtein

In [170]:
PRINT_LEN = 50

In [ ]:
def get_accuracy(pred, label):
    corrects = 0
    for i in range(len(pred)):
        if pred[i] == label[i]:
            corrects += 1
    acc = corrects / len(pred)
    distance = Levenshtein.distance(pred, label)
    print(f'Accuracy: {acc:.3f}')
    return acc, distance

In [172]:
data_path = 'texts/WarAndPeace.txt'
with open(data_path, 'r') as f:
    corpus = f.read()
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
corpus = corpus.lower().replace('\n', ' ').replace('\t', ' ')
corpus = ''.join([x for x in corpus if x in alphabet])
corpus = ' '.join(corpus.split())
corpus[:PRINT_LEN]


'война и мир самый известный роман льва николаевича'

### 1. Базовый частотный метод

In [173]:
unigrams = Counter(corpus).most_common()
unigrams[:5]

[(' ', 103408), ('о', 61282), ('а', 45209), ('е', 42519), ('и', 35838)]

In [174]:
test = corpus[:10000]

unique_chars = list(set(test))
unique_chars_shuffled = unique_chars.copy()
random.Random(42).shuffle(unique_chars_shuffled)

encoder_dict = dict(zip(unique_chars, unique_chars_shuffled))
trans_table = str.maketrans(encoder_dict)

test_encoded = test.translate(trans_table)
encoded_unigrams = Counter(test_encoded).most_common()
print('true:   ', test[:PRINT_LEN])
print('encoded:', test_encoded[:PRINT_LEN])

true:    война и мир самый известный роман льва николаевича
encoded: пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё


In [175]:
sorted(list(set(corpus)))

[' ',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 'ё']

In [106]:
print('true:    ', test[:PRINT_LEN])
print('encoded: ', test_encoded[:PRINT_LEN])
for i in range(40):
    decoder_dict = dict(zip([x[0] for x in encoded_unigrams[:i + 1]], [x[0] for x in unigrams[:i + 1]]))
    trans_table = str.maketrans(decoder_dict)
    test_decoded = test_encoded.translate(trans_table)
    print(f'top {i + 1:2} :', test_decoded[:PRINT_LEN], end='   ')
    get_accuracy(test, test_decoded);

true:     война и мир самый известный роман льва николаевича
encoded:  пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё
top  1 : пгэбё   у ю оёучэ  цпсоибчэ югуёб жъпё б йгжёсп кё   Accuracy: 0.162  Levenstain distance: 8333
top  2 : поэбё   у ю оёучэ  цпсоибчэ юоуёб жъпё б йожёсп кё   Accuracy: 0.251  Levenstain distance: 7483
top  3 : поэбё   у ю оёучэ  цпаоибчэ юоуёб жъпё б йожёап кё   Accuracy: 0.251  Levenstain distance: 7488
top  4 : поэбе   у ю оеучэ  цпаоибчэ юоуеб жъпе б йожеап ке   Accuracy: 0.251  Levenstain distance: 7486
top  5 : поэие   у ю оеучэ  цпаоиичэ юоуеи жъпе и йожеап ке   Accuracy: 0.251  Levenstain distance: 7476
top  6 : поэие н уню оеучэ нцпаоиичэ юоуеи жъпе инйожеапнке   Accuracy: 0.251  Levenstain distance: 7471
top  7 : поэие н уню оеучэ нцпаотичэ юоуеи жъпе инйожеапнке   Accuracy: 0.301  Levenstain distance: 6981
top  8 : соэие н уню оеучэ нцсаотичэ юоуеи жъсе инйожеаснке   Accuracy: 0.301  Levenstain distance: 6980
top  9 : соэие н уню леучэ нцсалтичэ ю

In [107]:
decoder_dict

{'е': ' ',
 'г': 'о',
 'с': 'а',
 'ё': 'е',
 'б': 'и',
 ' ': 'н',
 'и': 'т',
 'п': 'с',
 'о': 'л',
 'ю': 'в',
 'ж': 'р',
 'й': 'к',
 'у': 'д',
 'з': 'м',
 'ы': 'у',
 'ф': 'п',
 'н': 'я',
 'ц': 'г',
 'ч': 'ь',
 'ъ': 'ы',
 'к': 'з',
 'р': 'б',
 'ш': 'ч',
 'э': 'й',
 'т': 'ж',
 'я': 'ш',
 'щ': 'х',
 'а': 'ю',
 'м': 'ц',
 'в': 'э',
 'ь': 'щ',
 'д': 'ф',
 'х': 'ё',
 'л': 'ъ'}

### 2. Биграммный частотный метод

In [108]:
bigrams = Counter(corpus[x:x+2] for x in range(len(corpus) - 1)).most_common()
print(f'bigrams: {len(bigrams)}')
bigrams[:5]

bigrams: 796


[('о ', 13316), ('и ', 11398), ('а ', 10596), ('е ', 10040), (' с', 9863)]

In [109]:
unique_chars = Counter(corpus[x:x+2] for x in range(len(corpus) - 1)).most_common() 
unique_chars = [x[0] for x in unique_chars]
unique_chars_shuffled = unique_chars.copy()
random.Random(42).shuffle(unique_chars_shuffled)

encoder_dict = dict(zip(unique_chars, unique_chars_shuffled))

test_encoded = ''.join([encoder_dict[test[i:i+2]] for i in range(0, len(test), 2)])
encoded_bigrams = Counter(test_encoded[x:x+2] for x in range(len(test_encoded) - 1)).most_common() 
print(test[:100])
print(test_encoded[:100])

In [110]:
print('true:    ', test[:PRINT_LEN])
print('encoded: ', test_encoded[:PRINT_LEN])
for i in range(20):
    decoder_dict = dict(zip([x[0] for x in encoded_bigrams[:i + 1]], [x[0] for x in bigrams[:i + 1]]))
    test_decoded = ''.join([decoder_dict[test_encoded[i:i+2]] if test_encoded[i:i+2] in decoder_dict else test_encoded[i:i+2] for i in range(0, len(test_encoded), 2)])
    print(f'top {i + 1:2} :', test_decoded[:PRINT_LEN], end='   ')
    get_accuracy(test, test_decoded);

true:     война и мир самый известный роман льва николаевича
encoded:  пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё
top  1 : пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё   Accuracy: 0.000  Levenstain distance: 8835
top  2 : пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё   Accuracy: 0.000  Levenstain distance: 8835
top  3 : пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё   Accuracy: 0.000  Levenstain distance: 8835
top  4 : пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё   Accuracy: 0.000  Levenstain distance: 8835
top  5 : пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё   Accuracy: 0.000  Levenstain distance: 8835
top  6 : пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё   Accuracy: 0.000  Levenstain distance: 8835
top  7 : пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё   Accuracy: 0.000  Levenstain distance: 8835
top  8 : пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё   Accuracy: 0.000  Levenstain distance: 8835
top  9 : пгэбёе еу юеоёучэе цпсоибчэею

In [111]:
decoder_dict

{'р': 'о ',
 'л': 'и ',
 'ц': 'а ',
 'д': 'по',
 'е': ' н',
 ' ': ' в',
 'а': ' о',
 'т': 'я ',
 'в': ' к',
 'з': ' и',
 'ы': 'ст',
 'ш': 'ал',
 'п': 'на',
 'й': 'го',
 'с': 'не'}

### 3. MCMC-сэмплированиe

In [112]:
import numpy as np
import pandas as pd

In [ ]:
test = 'роста как пишет мне мой староста донесенья покойерп она хорошей фамилии и богата всё что мне нужно и он с теми свободными и фамильярными грациозными движениями которые его отличали взял за руку фрейлину поцеловал ее и поцеловав помахал фрейлинскою рукой развалившись на креслах и глядя в сторону подождите сказала анна павловна соображая я нынче же поговорю болконский с лизой женой молодого болконского и может быть это уладится я в вашем семействе начну обучаться ремеслу старой девки гостиная анны павловны начала понемногу наполняться приехала высшая знать петербурга люди самые разнородные по возрастам и характерам но одинаковые по обществу в каком все жили приехала дочь князя василия красавица элен заехавшая за отцом чтобы с ним вместе ехать на праздник посланника она была в шифре и бальном платье приехала и известная как самая обворожительная женщина в петербурге молодая маленькая княгиня болконская прошлую зиму вышедшая замуж и теперь'


In [154]:
test = 'роста как пишет мне мой староста донесенья покойерп она хорошей фамилии и богата всё что мне нужно и он с теми свободными и фамильярными грациозными движениями которые его отличали взял за руку фрейлину поцеловал ее и поцеловав помахал фрейлинскою рукой развалившись на креслах и глядя в сторону подождите сказала анна павловна соображая я нынче же поговорю болконский с лизой женой молодого болконского и может быть это уладится я в вашем семействе начну обучаться ремеслу старой девки гостиная анны павловны начала понемногу наполняться приехала высшая знать петербурга люди самые разнородные по возрастам и характерам но одинаковые по обществу в каком все жили приехала дочь князя василия красавица элен заехавшая за отцом чтобы с ним вместе ехать на праздник посланника она была в шифре и бальном платье приехала и известная как самая обворожительная женщина в петербурге молодая маленькая княгиня болконская прошлую зиму вышедшая замуж и теперь'
test_encoded = 'ъм кзвэзэвшлрчкваычвамёв кзъм кзвомыч чыясвшмэмёчъшвмызвимъмрчёвхзалщллвлвдмгзкзвп нвекмваычвытьымвлвмыв вкчалв пмдмоыюалвлвхзалщясъыюалвгъзблмуыюалвопльчылсалвэмкмъючвчгмвмкщлезщлвпусщвузвътэтвхъчёщлытвшмбчщмпзщвччвлвшмбчщмпзпвшмазизщвхъчёщлы эмйвътэмёвъзупзщлпрл явызвэъч щзивлвгщсосвпв кмъмытвшмомьолкчв эзузщзвзыызвшзпщмпызв ммдъзьзсвсвыюыечвьчвшмгмпмъйвдмщэмы элёв вщлумёвьчымёвамщмомгмвдмщэмы эмгмвлвамьчквдюкявцкмвтщзолк свсвпвпзрчав чачё кпчвызеытвмдтезкя свъчач щтв кзъмёвочпэлвгм клызсвзыыювшзпщмпыювызезщзвшмычаымгтвызшмщыскя свшълчизщзвпю рзсвуызкявшчкчъдтъгзвщйолв заючвъзуымъмоыючвшмвпмуъз кзавлвизъзэкчъзавымвмолызэмпючвшмвмдфч кптвпвэзэмавп чвьлщлвшълчизщзвомеявэысусвпз лщлсвэъз зплбзвцщчывузчизпрзсвузвмкбмавекмдюв вылавпач кчвчизкявызвшъзуоылэвшм щзыылэзвмызвдющзвпврлхъчвлвдзщяымавшщзкячвшълчизщзвлвлупч кызсвэзэв зазсвмдпмъмьлкчщяызсвьчыфлызвпвшчкчъдтъгчвамщмозсвазщчыяэзсвэысглысвдмщэмы эзсвшъмрщтйвулатвпюрчорзсвузатьвлвкчшчъя'

In [113]:
# corpus = 'abcadaddegfc  abcada cbaddcfffff'

In [193]:
freq_dict = Counter(corpus)
freq_dict = Counter(corpus[x:x+2] for x in range(len(corpus) - 1))
tokens_count = np.array(list(freq_dict.values())).sum()
for key in freq_dict.keys():
    freq_dict[key] = np.log(freq_dict[key] / tokens_count)
    # freq_dict[key] = np.log(freq_dict[key]) # / tokens_count
    # freq_dict[key] = freq_dict[key]
freq_dict;


def change_text(text):
    token_1 = random.choice(vocab)[0]
    token_2 = random.choice(vocab)[0]
    encoder_dict = {
        token_1: token_2,
        token_2: token_1
        }
    trans_table = str.maketrans(encoder_dict)
    text_new = text.translate(trans_table)
    return text_new

def get_score(text, freq_dict):
    score = 0
    for i in range(len(text) - 1):
        cur_token = text[i:i + 2]
        score += freq_dict.get(cur_token, 0)
        # score += np.log(freq_dict.get(cur_token, 1))
    # return np.log(score)
    return score


num_iters = 40000

cur_text = test_encoded
best_text = test_encoded
best_score = get_score(cur_text, freq_dict)
print('true:', get_score(test, freq_dict))
print('true ', test[:50])
print('     ', test_encoded[:50])
for i in range(num_iters):
    new_text = change_text(cur_text)
    cur_score = get_score(new_text, freq_dict)
    if cur_score >= best_score:
        best_score = cur_score
        best_text = cur_text
    # elif np.log(random.random()) < cur_score - best_score:
    elif np.log(random.random()) < best_score - cur_score:
        cur_text = new_text
        
    if i % 2000 == 0:
        print(f'{i:5} {best_text[:50]} {best_score:.3f} {cur_score:.3f} {cur_score - best_score:.3f}', end=' ')
        get_accuracy(test, best_text);
print('true ', test[:50])
print('true ', best_text[:50])
# cur_text


true: 78586.85798152107
true  война и мир самый известный роман льва николаевича
      пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё
    0 пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё 41073.719 41073.719 0.000 Accuracy: 0.000  Levenstain distance: 8835.000
 2000 еьцпянонжоднйяжъцнотесйлпъцндьжяпныкеянпобьыясеоия 49373.997 33726.398 -15647.598 Accuracy: 0.000  Levenstain distance: 8924.000
 4000 еьцпянонжоднйяжъцнотесйлпъцндьжяпныкеянпобьыясеоия 49373.997 36839.619 -12534.378 Accuracy: 0.000  Levenstain distance: 8924.000
 6000 еьцпянонжоднйяжъцнотесйлпъцндьжяпныкеянпобьыясеоия 49373.997 23687.242 -25686.755 Accuracy: 0.000  Levenstain distance: 8924.000
 8000 ирхыщешеошдекщоюхешжи куыюхедрощыенлищеыштрнщ ишвщ 52646.030 36672.698 -15973.332 Accuracy: 0.015  Levenstain distance: 8738.000
10000 ирхыщешеошдекщоюхешжи куыюхедрощыенлищеыштрнщ ишвщ 52646.030 29890.976 -22755.054 Accuracy: 0.015  Levenstain distance: 8738.000
12000 цолкаэсэтсрэеатщлэсуцнечкщлэротакэдзцаэксыоданцсйа 

In [168]:
get_accuracy(test, best_text);

Accuracy: 0.277  Levenstain distance: 685


In [207]:
freq_dict = Counter(corpus)
freq_dict = Counter(corpus[x:x+2] for x in range(len(corpus) - 1))
tokens_count = np.array(list(freq_dict.values())).sum()
for key in freq_dict.keys():
    freq_dict[key] = np.log(freq_dict[key]) # / tokens_count
freq_dict;


def change_text(text):
    token_1 = random.choice(vocab)[0]
    token_2 = random.choice(vocab)[0]
    encoder_dict = {
        token_1: token_2,
        token_2: token_1
        }
    trans_table = str.maketrans(encoder_dict)
    text_new = text.translate(trans_table)
    return text_new

def get_score(text, freq_dict):
    score = 0
    for i in range(len(text) - 1):
        cur_token = text[i:i + 2]
        score += freq_dict.get(cur_token, 0)
    return score


num_iters = 10000

cur_text = test_encoded
best_text = cur_text = test_encoded
best_score = cur_score = get_score(cur_text, freq_dict)
print('true:', get_score(test, freq_dict))
print('true ', test[:50])
print('     ', test_encoded[:50])
for i in range(num_iters):
    new_text = change_text(cur_text)
    new_score = get_score(new_text, freq_dict)
    if new_score >= best_score:
        best_score = new_score
        best_text = new_text
    if new_score >= cur_score:
        cur_score = new_score
        cur_text = new_text
    elif np.log(random.random()) < (new_score - cur_score):
        cur_text = new_text
    if i % 2000 == 0:
        print(f'{i:5} {best_text[:50]} {new_score:.3f} {cur_score:.3f} {cur_score - new_score:.3f}', end=' ')
        get_accuracy(test, best_text);
print('true ', test[:50])
print('true ', best_text[:50])
# cur_text


true: 78586.85798152107
true  война и мир самый известный роман льва николаевича
      пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё
    0 пгэбёе еу юеоёучэе цпсоибчэеюгуёбежъпёеб йгжёсп кё 41569.641 41569.641 0.000 Accuracy: 0.000  Levenstain distance: 8829.000
 2000 война е мер самый езвистный роман льва неколаивеча 71627.801 77780.445 6152.644 Accuracy: 0.867  Levenstain distance: 1334.000
 4000 война и мир самый известный роман льва николаевича 76691.166 78586.858 1895.692 Accuracy: 1.000  Levenstain distance: 0.000


KeyboardInterrupt: 

In [213]:
test_2 = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'
test_2_dict = {elem: vocab[i] for i, elem in enumerate(set(test_2))}
trans_table = str.maketrans(test_2_dict)
test_2 = test_2.translate(trans_table)
test_2

'онхвшцсшцв вмошжтпъгхижсбшвхвшдтфмвшжтпъгхижсбшмолнмшкшзмтйтшнттщчожвршлтмтпсбшхойлтшдптфвмгмишнлтпоошцнойтшцсшцношн охгхвшдпгцвхижтшвшдтхкфвмошъглнвъгхижсбшщгххшугшдтнхо жоошфомцопмтошуг гжвошлкпнгшатмршлтжофжтшршжвфойтшжоштщочге'

In [324]:
freq_dict = Counter(corpus)
freq_dict = Counter(corpus[x:x+2] for x in range(len(corpus) - 1))
tokens_count = np.array(list(freq_dict.values())).sum()
for key in freq_dict.keys():
    freq_dict[key] = np.log(freq_dict[key]) # / tokens_count
freq_dict;


def change_text(text):
    token_1 = random.choice(vocab)[0]
    token_2 = random.choice(vocab)[0]
    encoder_dict = {
        token_1: token_2,
        token_2: token_1
        }
    trans_table = str.maketrans(encoder_dict)
    text_new = text.translate(trans_table)
    return text_new

def get_score(text, freq_dict):
    score = 0
    for i in range(len(text) - 1):
        cur_token = text[i:i + 2]
        score += freq_dict.get(cur_token, 0)
    return score


num_iters = 100000

best_text = cur_text = test_2
best_score = cur_score = get_score(cur_text, freq_dict)
for i in range(num_iters):
    new_text = change_text(cur_text)
    new_score = get_score(new_text, freq_dict)
    if new_score >= best_score:
        best_score = new_score
        best_text = new_text
    if new_score >= cur_score:
        cur_score = new_score
        cur_text = new_text
    elif np.log(random.random()) <= (new_score - cur_score):
        cur_text = new_text
    if i % 10000 == 0:
        print(f'{i:5} {best_text[:]}')
        # print(f'{i:5} {best_text[:50]} {new_score:.3f} {cur_score:.3f} {cur_score - new_score:.3f}', end=' ')
        # get_accuracy(test, best_text);
print('true ', best_text)
# cur_text


    0 онхвшцсшцв вмошжтпъгхижсбшвхвшдтфмвшжтпъгхижсбшмолнмшкшзмтйтшнттщеожвршлтмтпсбшхойлтшдптфвмгмишнлтпоошцнойтшцсшцношн охгхвшдпгцвхижтшвшдтхкфвмошъглнвъгхижсбшщгххшугшдтнхо жоошфомцопмтошуг гжвошлкпнгшатмршлтжофжтшршжвфойтшжоштщоегч
10000 если вд вимите норжальнды или почти норжальнды текст у этого сообщения которды легко прочитать скорее всего вд все смелали правильно и получите жаксижальнды балл за послемнее четвертое замание курса хотя конечно я ничего не обещаш
20000 если вд вимите норжальнды или почти норжальнды текст у этого сообщения которды легко прочитать скорее всего вд все смелали правильно и получите жаксижальнды балл за послемнее четвертое замание курса хотя конечно я ничего не обещаш
30000 если вд вимите норжальнды или почти норжальнды текст у этого сообщения которды легко прочитать скорее всего вд все смелали правильно и получите жаксижальнды балл за послемнее четвертое замание курса хотя конечно я ничего не обещаш
40000 если вд вимите норжальнды или почти норжальнды

In [316]:
random.random()

0.797619295202445

In [118]:
def get_score(text, freq_dict):
    score = 0
    for i in range(len(text)):
        # if text[i:i+2] in token2id and text[i+:i+4] in token2id:
        cur_token = text[i]
        score += freq_dict[cur_token]
    return np.log(score / i)
get_score(test, freq_dict)

-2.731934305799938

In [119]:
np.log(random.random())

-0.2650538135534482

In [120]:
(np.log(transition_matrix+1) / np.linalg.norm(np.log(transition_matrix+1), axis=1))

array([[0.        , 0.2113114 , 0.33949783, ..., 0.21099773, 0.27705022,
        0.09856901],
       [0.22592596, 0.06720113, 0.24309167, ..., 0.3505988 , 0.29993931,
        0.        ],
       [0.12293921, 0.18492354, 0.11375629, ..., 0.09085547, 0.2284306 ,
        0.        ],
       ...,
       [0.18694126, 0.        , 0.2123291 , ..., 0.17940647, 0.        ,
        0.        ],
       [0.2161536 , 0.01942548, 0.12410838, ..., 0.22723786, 0.19474447,
        0.        ],
       [0.14623088, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [121]:
random.random()

0.40244483433852585